# System

In [ ]:
if is_debug: print("\n---------- This notenook name: nb_system_function - Start ----------\n")

In [ ]:
medallion_name   = "System"
if is_debug: print(f"medallion_name: {medallion_name}")

In [ ]:
# Current session info
if is_debug: display(ss.getActiveSession())

## Print debug info

In [ ]:
if is_debug:
    def fn_print_debug_info(
        status
        , fn_name
        , par
    ):
        body = ""
        header = f"----- {status} ----- {fn_name}() (UTC: {dt.datetime.now()}) - Start -----"
        for k, v in par.items(): body += f"{k}: {str(v)}<br />"
        footer = f"----- {status} ----- {fn_name} - End -----"

        if status == "Success": color = "14a212"
        if status == "Warning": color = "c18d24"
        if status == "Danger": color = "bf0000"

        html = """<style>
            #header, #footer {
                color: #^color^;
                font-family: Consolas, monaco, monospace;
                font-size: 14px;
                font-weight: bold;
            }
            #body {
                font-family: Consolas, monaco, monospace;
                font-size: 14px;
            }
        </style>"""
        html =  html.replace("^color^", color)
        html += f"<div id=\"header\">{header}</div>"
        html += f"<div id=\"body\">{body}</div>"
        html += f"<div id=\"footer\">{footer}</div>"

        if status != "Success": displayHTML(html)

        return (True)

## Get now()

In [ ]:
def fn_get_now(format_):
    fn_name        = stk()[0][3]
    caller_fn_name = stk()[1].function.replace("<module>", "")
    if is_debug: par = {}

    try:
        tz = pt.timezone(global_parameter.time_zone_nb)

        match format_:
            case "datetime": now = dt.datetime.now(tz).strftime("%Y-%m-%d %H:%M:%S.%f")[:-3]
            case "date":     now = dt.datetime.now(tz).strftime("%Y-%m-%d")
            case "string":   now = dt.datetime.now(tz).strftime("%Y%m%d_%H%M%S%f")[:-3]
            case "year":     now = dt.datetime.now(tz).strftime("%Y")
            case "month":    now = dt.datetime.now(tz).strftime("%m")
            case "day":      now = dt.datetime.now(tz).strftime("%d")
            case "hour":     now = dt.datetime.now(tz).strftime("%H")
            case "minute":   now = dt.datetime.now(tz).strftime("%M")
            case "second":   now = dt.datetime.now(tz).strftime("%S")
            case _:          now = dt.datetime.now(tz)

        if is_debug:
            par["locals"] = locals()
            fn_print_debug_info("Success", fn_name, par)

        return (True, now)
    except Exception as ex:
        ex = str(ex)
        
        if is_debug:
            par["locals"] = locals()
            fn_print_debug_info("Danger", fn_name, ex)

        #notebookutils.notebook.exit("fn_get_now() is not working properly") ???

        return (False, ex)

# Log

## Create global dataframe pdf_log

In [ ]:
pdf_log = pd.DataFrame({
    "process_timestamp": pd.Series(dtype = "str")
    , "medallion_name": pd.Series(dtype = "str")
    , "source_name": pd.Series(dtype = "str")
    , "locals": pd.Series(dtype = "str")
    , "alert": pd.Series(dtype = "str")
    , "alert_description": pd.Series(dtype = "str")
    , "logged_datetime": pd.Series(dtype = "datetime64[ms]")
})

## Insert into local log

In [ ]:
def fn_local_log_insert(
    process_timestamp
    , medallion_name
    , source_name
    , locals_
    , alert
    , alert_description
):
    fn_name        = stk()[0][3]
    caller_fn_name = stk()[1].function.replace("<module>", "")
    if is_debug: par = {}
    
    try:
        global pdf_log

        logged_datetime = fn_get_now("datetime")[1]

        pdf_log_new_row = pd.DataFrame({
            "process_timestamp": pd.Series(dtype = "str")
            , "medallion_name": pd.Series(dtype = "str")
            , "source_name": pd.Series(dtype = "str")
            , "locals": pd.Series(dtype = "str")
            , "alert": pd.Series(dtype = "str")
            , "alert_description": pd.Series(dtype = "str")
            , "logged_datetime": pd.Series(dtype = "datetime64[ms]")
        })

        data_new_row = {"process_timestamp": process_timestamp
            , "medallion_name": medallion_name
            , "source_name": source_name
            , "locals": locals_
            , "alert": alert
            , "alert_description": alert_description
            , "logged_datetime": logged_datetime}

        pdf_log_new_row = pd.DataFrame([data_new_row])        
        pdf_log_new_row['logged_datetime'] = pdf_log_new_row['logged_datetime'].apply(lambda x: dt.datetime.strptime(x, '%Y-%m-%d %H:%M:%S.%f'))
        pdf_log = pd.concat([pdf_log, pdf_log_new_row])

        if is_debug:
            par["locals"] = locals()
            fn_print_debug_info("Success", fn_name, par)

        return (True, None)
    except Exception as ex:
        ex = str(ex)

        if alert_description == None: alert_description = ex

        if is_debug:
            par["locals"]    = locals()
            fn_print_debug_info("Danger", fn_name, par)

        return (False, ex)

## Insert into table "lh_log.log"

In [ ]:
def fn_lh_log_log_insert():
    fn_name        = stk()[0][3]
    caller_fn_name = stk()[1].function.replace("<module>", "")
    if is_debug: par = {}
    
    try:
        global pdf_log

        # Convert to sdf
        pdf_log['logged_datetime'] = pd.to_datetime(pdf_log['logged_datetime'], format = "%Y-%m-%d %H:%M:%S.%f")
        schema = st.StructType([
            st.StructField("process_timestamp", st.StringType())
            , st.StructField("medallion_name", st.StringType())
            , st.StructField("source_name", st.StringType())
            , st.StructField("locals", st.StringType())
            , st.StructField("alert", st.StringType())
            , st.StructField("alert_description", st.StringType())
            , st.StructField("logged_datetime", st.TimestampType())
        ])
        sdf_log = spark.createDataFrame(pdf_log, schema)

        # Format column logged_datetime
        sdf_log = sdf_log.withColumn(
            "logged_datetime"
            , sf.to_timestamp("logged_datetime", "yyyy-MM-dd HH:mm:ss.SSS")
        )
        
        # Append local log to lh_log.log
        sdf_log.write.format("delta")\
            .mode("append")\
            .save(f"{global_parameter.abfs_path_lh_log}/Tables/log")

        if is_debug:
            par["locals"] = locals()
            if "sdf_log" in locals(): par["sdf_log"] = sdf_log.show(n = 5)
            fn_print_debug_info("Success", fn_name, par)

        return (True, None)
    except Exception as ex:
        ex = str(ex)

        if is_debug:
            par["locals"]    = locals()
            fn_print_debug_info("Danger", fn_name, par)

        return (False, ex)

## locals() to JSON

In [ ]:
def fn_locals_to_json(loc):
    fn_name        = stk()[0][3]
    caller_fn_name = stk()[1].function.replace("<module>", "")
    if is_debug: par = {}
    
    try:        
        if is_debug: print(f"----- {fn_name} (loop locals()) - Start -----")
        
        locals_ = {}        
        for key, value in loc.items():
            if key != "locals_":                
                if is_debug:
                    print(f"key: {key} | value: {value} | value (type): {type(value)}")
                    print(f"---")

                if str(type(value)) in [
                    "<class 'pyspark.sql.dataframe.DataFrame'>"
                    , "<class 'pyspark.sql.types.StructType'>"
                    , "<class 'pandas.core.frame.DataFrame'>"
                    , "<class 'delta.tables.DeltaTable'>"
                    , "<class 'datetime.datetime'>"
                    , "<class 'tuple'>"
                    , "<class 're.Pattern'>"
                    , "<class 'pyspark.sql.column.Column'>"
                    , "<class 'pyspark.sql.types.Row'>"
                    , "<class 'notebookutils.mssparkutils.handlers.fsHandler.FileInfo'>"
                    , "<class 'list'>"
                    , "<class 'requests.models.Response'>"
                    , "<class 'module'>"
                ]:
                    value = str(value)
                
                locals_[key] = value        
        locals_ = j.dumps(locals_)

        alert = "Success"
        alert_description = ""

        if is_debug: print(f"----- {fn_name} (loop locals()) - End -----")

        return (alert, alert_description, locals_)
    except Exception as ex:
        alert             = "Danger"
        alert_description = str(ex)

        return (alert, alert_description, None)
    finally:
        if is_debug:
            par["locals"] = locals()
            fn_print_debug_info(alert, fn_name, par)
            del par
        
        if alert == "Danger":
            fn_local_log_insert(global_parameter.process_timestamp, medallion_name, fn_name, locals_, alert, alert_description)

## Insert "global_parameter" into local log

In [ ]:
def fn_local_log_insert_global_parameter():
    fn_name        = stk()[0][3]
    caller_fn_name = stk()[1].function.replace("<module>", "")
    if is_debug: par = {}
    
    try:
        str_global_parameter        = j.dumps(global_parameter.__dict__)
        dict_global_parameter       = j.loads(str_global_parameter)
        dict_global_parameter_count = len(dict_global_parameter)

        alert             = "Success"
        alert_description = f"Global parameter (count): {dict_global_parameter_count}"

        # fn_print_debug_info_123("Success", fn_name, par) # test "except"

        return (alert, alert_description)
    except Exception as ex:
        alert             = "Danger"
        alert_description = str(ex)

        return (alert, alert_description)
    finally:
        if is_debug:
            par["locals"] = locals()
            fn_print_debug_info(alert, fn_name, par)
            del par
        
        del str_global_parameter
        fn_local_log_insert(global_parameter.process_timestamp, medallion_name, fn_name, fn_locals_to_json(locals())[2], alert, alert_description)

# Lakehouse

## Execute PySpark SQL code

In [ ]:
def fn_execute_spark_sql(sql_code):
    fn_name        = stk()[0][3]
    caller_fn_name = stk()[1].function.replace("<module>", "")
    if is_debug: par = {}

    try:
        sdf               = spark.sql(sql_code)
        sdf_count         = sdf.count()
        alert             = "Success"
        alert_description = f"Rows (count): {sdf_count}"

        # fn_print_debug_info_123("Success", fn_name, par) # test "except"

        return (alert, alert_description, sdf, sdf_count)
    except Exception as ex:
        alert             = "Danger"
        alert_description = str(ex)

        return (alert, alert_description, None, None)
    finally:
        if is_debug:
            par["locals"] = locals()
            if "sdf" in locals() and sdf != None: par["sdf"] = sdf.show(n = 5)
            fn_print_debug_info(alert, fn_name, par)
            del par

        fn_local_log_insert(global_parameter.process_timestamp, medallion_name, fn_name, fn_locals_to_json(locals())[2], alert, alert_description)

# Log

## Clean up "lh_log.log"

In [ ]:
# Depends on fn_execute_spark_sql()

def fn_lh_log_log_cleanup(days_to_keep_log):
    fn_name        = stk()[0][3]
    caller_fn_name = stk()[1].function.replace("<module>", "")
    if is_debug: par = {}

    try:
        now = fn_get_now("date")[1]
        now_str = dt.datetime.strptime(now, "%Y-%m-%d")
        logged_datetime = now_str - dt.timedelta(days = int(days_to_keep_log))
        sql_query = f"""DELETE FROM delta.`{global_parameter.abfs_path_lh_log}/Tables/log`
WHERE logged_datetime < '{str(logged_datetime)}';"""

        rv_execute_spark_sql = fn_execute_spark_sql(sql_query)
        alert                = rv_execute_spark_sql[0]
        sdf                  = rv_execute_spark_sql[2]
        sdf_count            = sdf.collect()[0][0]

        alert             = "Success"
        alert_description = f"Log deleted before (date): {str(logged_datetime)}; Rows (count): {sdf_count}"

        # fn_print_debug_info_123("Success", fn_name, par) # test "except"

        return (alert, alert_description, sdf_count)
    except Exception as ex:
        alert             = "Danger"
        alert_description = str(ex)

        return (alert, alert_description, None)
    finally:
        if is_debug:
            par["locals"] = locals()
            if "sdf" in locals(): par["sdf"] = sdf.show(n = 5)
            fn_print_debug_info(alert, fn_name, par)
            del par
        
        fn_local_log_insert(global_parameter.process_timestamp, medallion_name, fn_name, fn_locals_to_json(locals())[2], alert, alert_description)

# File System

## Does file exist

In [ ]:
def fn_does_file_exist(
    file_name
    , dict_parameter
):
    fn_name        = stk()[0][3]
    caller_fn_name = stk()[1].function.replace("<module>", "")
    if is_debug: par = {}

    try:
        if notebookutils.fs.exists(file_name) == True:
            does_file_exist   = True
            alert             = "Success"
            alert_description = f"File \"{file_name}\" exists."
        else:
            does_file_exist   = False
            alert             = "Warning"
            alert_description = f"File \"{file_name}\" does not exist."

            # fn_print_debug_info_123("Success", fn_name, par) # test "except"

        return (alert, alert_description, does_file_exist)
    except Exception as ex:
        alert             = "Danger"
        alert_description = str(ex)

        return (alert, alert_description, None)
    finally:
        if is_debug:
            par["locals"] = locals()
            fn_print_debug_info(alert, fn_name, par)
            del par
        
        fn_local_log_insert(global_parameter.process_timestamp, medallion_name, fn_name, fn_locals_to_json(locals())[2], alert, alert_description)

## Copy file

In [ ]:
def fn_copy_file(
    src,
    dstn,
    dict_parameter
):
    fn_name        = stk()[0][3]
    caller_fn_name = stk()[1].function.replace("<module>", "")
    if is_debug: par = {}

    try:
        if notebookutils.fs.exists(src) == True:
            notebookutils.fs.fastcp(src, dstn, recurse = True)
            alert             = "Success"
            alert_description = f"File \"{src}\" is copied successfully."
        else:
            alert             = "Warning"
            alert_description = f"File \"{src}\" does not exist."

        # fn_print_debug_info_123("Success", fn_name, par) # test "except"

        return (alert, alert_description)
    except Exception as ex:
        alert             = "Danger"
        alert_description = str(ex)

        return (alert, alert_description)
    finally:
        if is_debug:
            par["locals"] = locals()
            fn_print_debug_info(alert, fn_name, par)
            del par
        
        fn_local_log_insert(global_parameter.process_timestamp, medallion_name, fn_name, fn_locals_to_json(locals())[2], alert, alert_description)

## Delete file

In [ ]:
def fn_delete_file(
    file_name,
    dict_parameter
):
    fn_name        = stk()[0][3]
    caller_fn_name = stk()[1].function.replace("<module>", "")
    if is_debug: par = {}

    try:
        if notebookutils.fs.exists(file_name):
            notebookutils.fs.rm(file_name)
            alert             = "Success"
            alert_description = f"File \"{file_name}\" is deleted successfully."
        else:
            alert             = "Warning"
            alert_description = f"File \"{file_name}\" does not exist."

        # fn_print_debug_info_123("Success", fn_name, par) # test "except"

        return (alert, alert_description)
    except Exception as ex:
        alert             = "Danger"
        alert_description = str(ex)

        return (alert, alert_description)
    finally:
        if is_debug:
            par["locals"] = locals()
            fn_print_debug_info(alert, fn_name, par)
            del par
        
        fn_local_log_insert(global_parameter.process_timestamp, medallion_name, fn_name, fn_locals_to_json(locals())[2], alert, alert_description)

## Move file

In [ ]:
def fn_move_file(
    src,
    dstn,
    dict_parameter
):
    fn_name        = stk()[0][3]
    caller_fn_name = stk()[1].function.replace("<module>", "")
    if is_debug: par = {}

    try:        
        if notebookutils.fs.exists(src) == True:
            if src != dstn:
                notebookutils.fs.mv(src, dstn, create_path = True, overwrite = True)
                alert             = "Success"
                alert_description = f"File \"{src}\" is moved successfully."
            else:
                alert             = "Warning"
                alert_description = f"Source and destination (\"{src}\") are the same. File not moved."            
        else:
            alert             = "Warning"
            alert_description = f"File \"{src}\" does not exist."

        # fn_print_debug_info_123("Success", fn_name, par) # test "except"

        return (alert, alert_description)
    except Exception as ex:
        alert             = "Danger"
        alert_description = str(ex)

        return (alert, alert_description)
    finally:
        if is_debug:
            par["locals"] = locals()
            fn_print_debug_info(alert, fn_name, par)
            del par
        
        fn_local_log_insert(global_parameter.process_timestamp, medallion_name, fn_name, fn_locals_to_json(locals())[2], alert, alert_description)

## Delete folder and all the files and folders in this folder recursively

In [ ]:
# Delete folder and everything in it
def fn_delete_folder(folder_name):
    fn_name        = stk()[0][3]
    caller_fn_name = stk()[1].function.replace("<module>", "")
    if is_debug: par = {}

    try:            
        if notebookutils.fs.exists(folder_name):
            notebookutils.fs.rm(folder_name, recurse = True)
            alert             = "Success"
            alert_description = f"Folder \"{folder_name}\" is deleted successfully."
        else:
            alert             = "Warning"
            alert_description = f"Folder \"{folder_name}\" does not exist."

        # fn_print_debug_info_123("Success", fn_name, par) # test "except"

        return (alert, alert_description)
    except Exception as ex:
        alert             = "Danger"
        alert_description = str(ex)

        return (alert, alert_description)
    finally:
        if is_debug:
            par["locals"] = locals()
            fn_print_debug_info(alert, fn_name, par)
            del par
        
        fn_local_log_insert(global_parameter.process_timestamp, medallion_name, fn_name, fn_locals_to_json(locals())[2], alert, alert_description)

## Delete all files and folders in folder

In [ ]:
# Doesn't delete the folder itself
def fn_delete_all_in_folder(folder_name):
    fn_name        = stk()[0][3]
    caller_fn_name = stk()[1].function.replace("<module>", "")
    if is_debug: par = {}

    try:        
        if notebookutils.fs.exists(folder_name):
            files = notebookutils.fs.ls(folder_name)
            for file_ in files: notebookutils.fs.rm(file_.path, True)

            files = str(files)
            file_ = "<Deleted>"
            
            alert             = "Success"
            alert_description = ""
        else:
            alert             = "Warning"
            alert_description = f"Folder \"{folder_name}\" does not exist."

        # fn_print_debug_info_123("Success", fn_name, par) # test "except"

        return (alert, alert_description)
    except Exception as ex:
        alert             = "Danger"
        alert_description = str(ex)

        return (alert, alert_description)
    finally:
        if is_debug:
            par["locals"] = locals()
            fn_print_debug_info(alert, fn_name, par)
            del par
        
        fn_local_log_insert(global_parameter.process_timestamp, medallion_name, fn_name, fn_locals_to_json(locals())[2], alert, alert_description)

## List folder

In [ ]:
def fn_list_folder(
    folder_name
    , dict_parameter
):
    fn_name        = stk()[0][3]
    caller_fn_name = stk()[1].function.replace("<module>", "")
    if is_debug: par = {}

    list_file       = []
    list_file_count = 0

    try:
        if notebookutils.fs.exists(folder_name):
            list_file = notebookutils.fs.ls(folder_name)
            list_file_count = len(list_file)
            alert             = "Success"
            alert_description = f"List file (count) {list_file_count}"
        else:
            alert             = "Warning"
            alert_description = f"Folder \"{folder_name}\" does not exist."

        # fn_print_debug_info_123("Success", fn_name, par) # test "except"

        return (alert, alert_description, list_file, list_file_count)
    except Exception as ex:
        alert             = "Danger"
        alert_description = str(ex)

        return (alert, alert_description, None, None)
    finally:
        if is_debug:
            par["locals"] = locals()
            fn_print_debug_info(alert, fn_name, par)
            del par
        
        fn_local_log_insert(global_parameter.process_timestamp, medallion_name, fn_name, fn_locals_to_json(locals())[2], alert, alert_description)

# Dataframe, Temp View

## Generate dynamic "StructType" list

In [ ]:
# Return Dynamic Python code to create new dataframe with proper data types
# Used to create new delta table
def fn_get_struct_type(sdf):
    fn_name        = stk()[0][3]
    caller_fn_name = stk()[1].function.replace("<module>", "")
    if is_debug:
        par = {}

    try:
        schema = "StructType(["

        for row in sdf.sort(sdf.sequence.asc()).collect():
            if row.sequence == 1: comma = " "
            else: comma = " , "

            schema += f"\n{comma}StructField(\"{row.column_name}\", {row.data_type}Type())"

        schema += "\n])"
        
        alert             = "Success"
        alert_description = f""

        # fn_print_debug_info_123("Success", fn_name, par) # test "except"

        return (alert, alert_description, schema)
    except Exception as ex:
        alert             = "Danger"
        alert_description = str(ex)

        return (alert, alert_description, None)
    finally:
        if is_debug:
            par["locals"] = locals()
            if "sdf" in locals(): par["sdf"] = sdf.show(n = 5)
            fn_print_debug_info(alert, fn_name, par)
            del par
        
        fn_local_log_insert(global_parameter.process_timestamp, medallion_name, fn_name, fn_locals_to_json(locals())[2], alert, alert_description)

## Generate select columns for extract

In [ ]:
# Return Dynamic Python code to apply aliases and cast to propre data type
# Used to select from dataframe and insert into delta table
# Columns renamed with alias and datatypes applied

def fn_get_select_column(
    sdf_name
    , sdf
):
    fn_name        = stk()[0][3]
    caller_fn_name = stk()[1].function.replace("<module>", "")
    if is_debug:
        par = {}

    try:
        column = sdf_name + ".select("

        for row in sdf.sort(sdf.sequence.asc()).collect():
            if row.sequence == 1: comma = ""
            else: comma = ","

            column += f"\n{comma} {sdf_name}[\"{row.column_name}\"].alias(\"{row.alias_name}\").cast(st.{row.data_type}Type())"

        column += ")"

        alert             = "Success"
        alert_description = f""

        # fn_print_debug_info_123("Success", fn_name, par) # test "except"

        return (alert, alert_description, column)
    except Exception as ex:
        alert             = "Danger"
        alert_description = str(ex)

        return (alert, alert_description, None)
    finally:
        if is_debug:
            par["locals"] = locals()
            if "sdf" in locals(): par["sdf"] = sdf.show(n = 5)
            fn_print_debug_info(alert, fn_name, par)
            del par
        
        fn_local_log_insert(global_parameter.process_timestamp, medallion_name, fn_name, fn_locals_to_json(locals())[2], alert, alert_description)

## Save SDF in OneLake as table

In [ ]:
def fn_save_sdf_as_table(
    sdf
    , format_
    , mode
    , path
    , dict_parameter
):
    fn_name        = stk()[0][3]
    caller_fn_name = stk()[1].function.replace("<module>", "")
    if is_debug: par = {}

    try:
        # If overwrite the table, overwrite the schema too
        if mode == "overwrite": option = ".option(\"overwriteSchema\", \"True\")"
        else:                   option = ""

        # Set dynamic code
        code = f"sdf.write.format(\"{format_}\").mode(\"{mode}\"){option}.save(\"{path}\")"

        # Create table (execute dynamic code)
        eval(code)


        # Set "table_name"
        if dict_parameter["technology"] == "SQL Server":
            server_name   = dict_parameter["server_name_clean"]
            database_name = dict_parameter["database_name_clean"]
            schema_name   = dict_parameter["schema_name_clean"]
            table_name    = dict_parameter["table_name_clean"]
            table         = f"{server_name}_{database_name}_{schema_name}_{table_name}"

        if dict_parameter["technology"] == "Lakehouse":
            lakehouse_name = dict_parameter["lakehouse_name_clean"]
            table_name     = dict_parameter["table_name_clean"]
            table          = f"{lakehouse_name}.{table_name}"

        if dict_parameter["technology"] == "Excel":
            technology     = dict_parameter["technology_clean"]
            folder_name    = dict_parameter["folder_name_clean"]
            file_name      = dict_parameter["file_name_clean"]
            worksheet_name = dict_parameter["worksheet_name_clean"]
            table          = f"{technology}_{folder_name}_{file_name}_{worksheet_name}"

        if dict_parameter["technology"] == "CSV":
            technology     = dict_parameter["technology_clean"]
            folder_name    = dict_parameter["folder_name_clean"]
            file_name      = dict_parameter["file_name_clean"]
            table          = f"{technology}_{folder_name}_{file_name}"

        if dict_parameter["technology"] == "JSON":
            technology     = dict_parameter["technology_clean"]
            folder_name    = dict_parameter["folder_name_clean"]
            file_name      = dict_parameter["file_name_clean"]
            table          = f"{technology}_{folder_name}_{file_name}"

        table = table.replace("__", "_")

        alert             = "Success"
        alert_description = f"Table \"{table}\" saved successfully."

        # fn_print_debug_info_123("Success", fn_name, par) # test "except"

        return (alert, alert_description)
    except Exception as ex:
        alert             = "Danger"
        alert_description = str(ex)

        return (alert, alert_description)
    finally:
        if is_debug:
            par["locals"] = locals()
            fn_print_debug_info(alert, fn_name, par)
            del par
        
        fn_local_log_insert(global_parameter.process_timestamp, medallion_name, fn_name, fn_locals_to_json(locals())[2], alert, alert_description)

## Get existing dataframe list

In [ ]:
# Get list of the existing dataframes

def fn_get_dataframe_list():
    fn_name        = stk()[0][3]
    caller_fn_name = stk()[1].function.replace("<module>", "")
    if is_debug: par = {}

    try:
        dataframe_list = [k for (k, v) in globals().items() if isinstance(v, df)]
        dataframe_list_count = len(dataframe_list)

        alert             = "Success"
        alert_description = f"Dataframes (count): {dataframe_list_count}"

        # fn_print_debug_info_123("Success", fn_name, par) # test "except"

        return (alert, alert_description, dataframe_list, dataframe_list_count)
    except Exception as ex:
        alert             = "Danger"
        alert_description = str(ex)

        return (alert, alert_description, None, None)
    finally:
        if is_debug:
            par["locals"] = locals()
            fn_print_debug_info(alert, fn_name, par)
            del par
        
        fn_local_log_insert(global_parameter.process_timestamp, medallion_name, fn_name, fn_locals_to_json(locals())[2], alert, alert_description)

## Get existing temp view list

In [ ]:
# Get list of the existing dataframes

def fn_get_temp_view_list():
    fn_name        = stk()[0][3]
    caller_fn_name = stk()[1].function.replace("<module>", "")
    if is_debug: par = {}

    try:
        view_list = fn_execute_spark_sql(f"SHOW VIEWS;")[2]
        view_list_count = view_list.count()

        alert             = "Success"
        alert_description = f"Views (count): {view_list_count}"

        # fn_print_debug_info_123("Success", fn_name, par) # test "except"

        return (alert, alert_description, view_list, view_list_count)
    except Exception as ex:
        alert             = "Danger"
        alert_description = str(ex)

        return (alert, alert_description, None, None)
    finally:
        if is_debug:
            par["locals"] = locals()
            fn_print_debug_info(alert, fn_name, par)
            del par
        
        fn_local_log_insert(global_parameter.process_timestamp, medallion_name, fn_name, fn_locals_to_json(locals())[2], alert, alert_description)

# String

## Clean all not alphanumericals in single SDF column

In [ ]:
def fn_replace_all_not_alphanumericals_in_single_column_in_sdf(
    sdf
    , col_name
    , replace_with
):
    fn_name        = stk()[0][3]
    caller_fn_name = stk()[1].function.replace("<module>", "")
    if is_debug: par = {}

    try:
        pdf = sdf.toPandas()
        pdf = pdf.replace(
            to_replace = {col_name: "\W+"},
            value      = replace_with,
            regex      = True
        )
        sdf = spark.createDataFrame(pdf)
        sdf_count = sdf.count()

        alert             = "Success"
        alert_description = f"Not alphanumericals in column \"{col_name}\" successfully replaced with \"{replace_with}\""

        # fn_print_debug_info_123("Success", fn_name, par) # test "except"

        return (alert, alert_description, sdf, sdf_count)
    except Exception as ex:
        alert             = "Danger"
        alert_description = str(ex)

        return (alert, alert_description, None, None)
    finally:
        if is_debug:
            par["locals"] = locals()
            fn_print_debug_info(alert, fn_name, par)
            del par
        
        fn_local_log_insert(global_parameter.process_timestamp, medallion_name, fn_name, fn_locals_to_json(locals())[2], alert, alert_description)

## Clean all not alphanumericals in string

In [ ]:
def fn_clean_not_alphanumeric_in_string(
    string
    , replace_with
):
    fn_name        = stk()[0][3]
    caller_fn_name = stk()[1].function.replace("<module>", "")
    if is_debug: par = {}

    try:
        reg_ex = re.compile("\W+")
        output = reg_ex.sub(replace_with, string).strip()

        alert             = "Success"
        alert_description = f"Not alphanumericals in string \"{string}\" successfully replaced with \"{replace_with}\""

        # fn_print_debug_info_123("Success", fn_name, par) # test "except"

        return (alert, alert_description, output)
    except Exception as ex:
        alert             = "Danger"
        alert_description = str(ex)

        return (alert, alert_description, None)
    finally:
        if is_debug:
            par["locals"] = locals()
            fn_print_debug_info(alert, fn_name, par)
            del par
        
        fn_local_log_insert(global_parameter.process_timestamp, medallion_name, fn_name, fn_locals_to_json(locals())[2], alert, alert_description)

## Clean accents in SDF column

In [ ]:
# Clean accents in single sdf column

def fn_clean_accent_in_sdf_column_make_trans():
    matching_string = ""
    replace_string = ""

    for i in range(ord(" "), sys.maxunicode):
        name = ucd.name(chr(i), "")
        if "WITH" in name:
            try:
                base = ucd.lookup(name.split(" WITH")[0])
                matching_string += chr(i)
                replace_string += base
            except KeyError:
                pass

    return matching_string, replace_string

def fn_clean_accent_in_sdf_column(column_name):
    fn_name        = stk()[0][3]
    caller_fn_name = stk()[1].function.replace("<module>", "")
    if is_debug: par = {}

    try:
        matching_string, replace_string = fn_clean_accent_in_sdf_column_make_trans()
        
        alert             = "Success"
        alert_description = f"Accents in column {column_name} are cleaned successfully."

        output = sf.translate(
            sf.regexp_replace(column_name, "\p{M}", "")
            , matching_string, replace_string
        ).alias(column_name)

        # fn_print_debug_info_123("Success", fn_name, par) # test "except"

        return (alert, alert_description, output)
    except Exception as ex:
        alert             = "Danger"
        alert_description = str(ex)

        return (alert, alert_description, None)
    finally:
        if is_debug:
            par["locals"] = locals()
            fn_print_debug_info(alert, fn_name, par)
            del par
        
        del matching_string
        del replace_string
        del output
        fn_local_log_insert(global_parameter.process_timestamp, medallion_name, fn_name, fn_locals_to_json(locals())[2], alert, alert_description)

## Clean accents in string

In [ ]:
# éèàç --> eeac

def fn_clean_accent_in_string(string):
    fn_name        = stk()[0][3]
    caller_fn_name = stk()[1].function.replace("<module>", "")
    if is_debug: par = {}

    try:
        output = "".join(c for c in ucd.normalize("NFD", string) if ucd.category(c) != "Mn")

        alert             = "Success"
        alert_description = f"Accents in string \"{string}\" are cleaned successfully."

        # fn_print_debug_info_123("Success", fn_name, par) # test "except"

        return (alert, alert_description, output)
    except Exception as ex:
        alert             = "Danger"
        alert_description = str(ex)

        return (alert, alert_description, None)
    finally:
        if is_debug:
            par["locals"] = locals()
            fn_print_debug_info(alert, fn_name, par)
            del par
        
        fn_local_log_insert(global_parameter.process_timestamp, medallion_name, fn_name, fn_locals_to_json(locals())[2], alert, alert_description)

## Replace in all the collumns in SDF

In [ ]:
def fn_replace_in_all_columns_in_sdf(
    sdf
    , replace_what
    , replace_with
):
    fn_name        = stk()[0][3]
    caller_fn_name = stk()[1].function.replace("<module>", "")
    if is_debug: par = {}

    try:
        cols = sdf.columns        
        for c in cols:
            sdf = sdf.withColumn(
                c
                , sf.when((sf.col(c) == replace_what), replace_with).otherwise(sf.col(c))
            )

        sdf_count = sdf.count()

        alert             = "Success"
        alert_description = f"\"{replace_what}\" is successfully replaced with \"{replace_with}\" in all columns."

        # fn_print_debug_info_123("Success", fn_name, par) # test "except"

        return (alert, alert_description, sdf, sdf_count)
    except Exception as ex:
        alert             = "Danger"
        alert_description = str(ex)

        return (alert, alert_description, None, None)
    finally:
        if is_debug:
            par["locals"] = locals()
            if "sdf" in locals(): par["sdf"] = sdf.show(n = 5)
            fn_print_debug_info(alert, fn_name, par)
            del par
     
        fn_local_log_insert(global_parameter.process_timestamp, medallion_name, fn_name, fn_locals_to_json(locals())[2], alert, alert_description)

## Replace in single collumn in SDF

In [ ]:
def fn_replace_in_single_column_in_sdf(
    sdf
    , column_name
    , replace_what
    , replace_with
):
    fn_name        = stk()[0][3]
    caller_fn_name = stk()[1].function.replace("<module>", "")
    if is_debug: par = {}

    try:
        sdf = sdf.withColumn(
            column_name
            , sf.regexp_replace(sf.col(column_name), replace_what, replace_with)
        )

        sdf_count = sdf.count()

        alert             = "Success"
        alert_description = f"\"{replace_what}\" is successfully replaced with \"{replace_with}\" in column \"{column_name}\"."

        # fn_print_debug_info_123("Success", fn_name, par) # test "except"

        return (alert, alert_description, sdf, sdf_count)
    except Exception as ex:
        alert             = "Danger"
        alert_description = str(ex)

        return (alert, alert_description, None, None)
    finally:
        if is_debug:
            par["locals"] = locals()
            if "sdf" in locals(): par["sdf"] = sdf.show(n = 5)
            fn_print_debug_info(alert, fn_name, par)
            del par
        
        fn_local_log_insert(global_parameter.process_timestamp, medallion_name, fn_name, fn_locals_to_json(locals())[2], alert, alert_description)

# API

## Mask Bearer Token

In [ ]:
# Mask Bearer Token before insert in log

def fn_mask_bearer_token(mask):
    # Mask dict_parameter
    if str(type(mask)) == "<class 'dict'>" and "headers" in mask:
        headers = mask["headers"]

        if "Authorization" in headers:
            authorization = headers["Authorization"]
            bearer = authorization[:7]
            authorization = f"{bearer}[REDACTED]"
            headers.update({"Authorization": authorization})
            mask.update({"headers": headers})
    
    # Mask request_code
    if str(type(mask)) == "<class 'str'>":
        index = mask.find("Bearer ")
        if index != -1:            
            authorization = mask[index:]
            end = authorization.find("'")
            replace_what = mask[(index + 7):(end + index)]
            mask = mask.replace(replace_what, "[REDACTED]")

    # Mask headers
    if "Authorization" in mask:
        authorization = mask["Authorization"]
        bearer = authorization[:7]
        authorization = f"{bearer}[REDACTED]"
        mask.update({"Authorization": authorization})

    return mask

## Execute API request

In [ ]:
# method: GET, POST
# response_format: JSON, XML
# dict_parameter
#  GET
#    url     String "https://api.github.com/events"
#    headers Dict   {"Content-Type": "application/json", "User-Agent": "zPL Concept", "Authorization": "Bearer 123"} # Bearer Token
#    params  Dict   {"id": 4}
#    auth    Tuple  ("Username", "Password") # Basic
#    timeout Int    5
#  POST
#    url     String "https://api.github.com/events"
#    headers Dict	
#    data    Dict
#    json    Dict
#    auth    Tuple
#    timeout Int    5

def fn_api_request(
    method
    , response_format
    , dict_parameter
):
    fn_name        = stk()[0][3]
    caller_fn_name = stk()[1].function.replace("<module>", "")
    if is_debug: par = {}

    try:
        alert = "Danger"

        arguments = ""
        for key, val in dict_parameter.items():
            if str(type(val)) == "<class 'str'>": val = f"\"{val}\""
            arguments += f"{key} = {val}, "
        arguments = arguments[:-2]
        
        if   method == "GET":    request_code = f"req.get({arguments})"
        elif method == "POST":   request_code = f"req.post({arguments})"
        elif method == "DELETE": request_code = f"req.delete({arguments})"

        response             = eval(request_code)
        is_response_ok       = response.ok
        response_status_code = response.status_code
        response_reason      = response.reason
        response_status      = f"{response_status_code} ({response_reason}"
        response.raise_for_status()

        if is_response_ok:
            alert             = "Success"
            alert_description = f"Response status: {response_status})"
            if response_format == "JSON":     return_value = (f"{response_status})", response.json())
            if response_format == "XML":      return_value = (f"{response_status})", response.text)
            if response_format == "Response": return_value = (f"{response_status})", response)
        else:    
            alert_description = f"Response status: {response_status}"

        return return_value
            
    except req.exceptions.HTTPError as http_err:
        alert             = "Danger"
        alert_description = f"{response_status}: HTTP Error)"
        return_value      = (f"{response_status}: HTTP Error)", http_err)

        return return_value
    except req.exceptions.ConnectionError as conn_err:
        alert             = "Danger"
        alert_description = f"{response_status}: Connection Error)"
        return_value      = (f"{response_status}: Connection Error)", conn_err)

        return return_value
    except req.exceptions.Timeout as timeout:
        alert             = "Danger"
        alert_description = f"{response_status}: Timeout)"
        return_value      = (f"{response_status}: Timeout)", timeout)

        return return_value
    except req.exceptions.RequestException as req_ex:
        alert             = "Danger"
        alert_description = f"{response_status}: Request Exception)"
        return_value      = (f"{response_status}: Request Exception)", req_ex)

        return return_value
    finally:
        if is_debug:
            par["locals"] = locals()
            if "sdf" in locals(): par["sdf"] = sdf.show(n = 5)
            fn_print_debug_info(alert, fn_name, par)
            del par

        # Keep only the first element of "return_value" in the log
        if response_format == "JSON":
            list_return_value = list(return_value)
            list_return_value[1] = list_return_value[1][:1]
            return_value = tuple(list_return_value)
            del list_return_value

        # Keep only the first 1000 characters of "return_value" in the log
        if response_format == "XML":
            list_return_value = list(return_value)
            list_return_value[1] = f"{list_return_value[1][:1000]}..."
            return_value = tuple(list_return_value)
            del list_return_value

        del key
        del val
        del arguments

        # Mask Bearer Token
        dict_parameter = fn_mask_bearer_token(dict_parameter)
        request_code = fn_mask_bearer_token(request_code)
        
        fn_local_log_insert(global_parameter.process_timestamp, medallion_name, fn_name, fn_locals_to_json(locals())[2], alert, alert_description)

In [ ]:
if is_debug: print("\n---------- This notenook name: nb_system_function - End ----------\n")